In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESFvsCNS_WedgeFlow");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 15; Grid Count = 15; Path = \\dc3\userspace\sebastian\cluster\XESFvsCNS_WedgeFlow }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-23 12:15:47Z

In [4]:
var sessions = database.Sessions;
sessions

#0: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax_agg0.3_RESTART12	3/23/2024 2:14:34 PM	a4ae7688...
#1: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_isUniform_False_ref10*	3/24/2024 10:24:29 AM	c5c39c1c...
#2: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_isUniform_False_ref6*	3/24/2024 10:21:16 AM	97aa26e9...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/23/2024 3:31:59 PM	0edcfdf3...
#4: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/23/2024 3:32:30 PM	d9b84916...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/23/2024 3:26:15 PM	df3ffe04...
#6: IBMWedgeFlow	IBMWedgeFlow_p1_xCells15_yCells10_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax_agg0.3_RESTART12*	3/23/2024 3:57:22 PM	7bf736d4...
#7: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/23/2024 3:10:44 PM	e6c497cc...
#8: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/23/2024 3:0

In [5]:
MatrixAssembler aaa = new MatrixAssembler(12);
var si = database.Sessions.Pick(12); // pick a session to compare
var ti = si.Timesteps;
si.ID

1b688a8a-bf70-4300-abab-4d6070b074d1

Error: System.ArgumentNullException: Value cannot be null. (Parameter 'path')
   at System.ArgumentNullException.Throw(String paramName)
   at System.ArgumentNullException.ThrowIfNull(Object argument, String paramName)
   at System.ArgumentException.ThrowNullOrEmptyException(String argument, String paramName)
   at System.IO.Strategies.FileStreamHelpers.ValidateArguments(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options, Int64 preallocationSize)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share)
   at BoSSS.Application.BoSSSpad.ResidualLog.InitializeReaderAndValues() in C:\experimental\public\src\L4-application\BoSSSpad\ResidualLog.cs:line 136
   at BoSSS.Application.BoSSSpad.ResidualLog.ReadResiduals() in C:\experimental\public\src\L4-application\BoSSSpad\ResidualLog.cs:line 201
   at BoSSS.Application.BoSSSpad.ResidualLog..ctor(ISessionInfo session, String norm, String[] variable

In [6]:
si.Export().Do() //export the plot

C:\Users\sebastian\AppData\Local\BoSSS\plots\sessions\IBMWedgeFlow__IBMWedgeFlow_p1_xCells60_yCells40_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax_agg0.3_RESTART12__1b688a8a-bf70-4300-abab-4d6070b074d1

In [7]:
var flds = si.Timesteps.Last().Fields;
var tp= new Tecplot(flds.Last().GridDat,2);
tp.PlotFields(si.Name,0,flds);


## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the WedgeFlow ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=2$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [9]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = si.Timesteps.Last().GetField("h");
var enthalpy_exact = (DGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

Compute the enthalpy error

In [10]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2Norm();
var EE = enthalpy_end.L2Error(enthalpy_exact)/EEN;
EE

0.22081460140149398